In [1]:
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules, fpgrowth
from mlxtend.preprocessing import TransactionEncoder
import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
try:
    df = pd.read_excel("OnlineRetail.xlsx")
    print("Dataset cargado exitosamente.")
    print("Dimensiones iniciales:", df.shape)
except FileNotFoundError:
    print("Error: Asegúrate de que el archivo 'OnlineRetail.xlsx' esté en el mismo directorio que tu notebook.")


print("Primeras filas del dataset:")
print(df.head())

Dataset cargado exitosamente.
Dimensiones iniciales: (541909, 8)
Primeras filas del dataset:
  InvoiceNo StockCode                          Description  Quantity  \
0    536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1    536365     71053                  WHITE METAL LANTERN         6   
2    536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3    536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4    536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   

          InvoiceDate  UnitPrice  CustomerID         Country  
0 2010-12-01 08:26:00       2.55     17850.0  United Kingdom  
1 2010-12-01 08:26:00       3.39     17850.0  United Kingdom  
2 2010-12-01 08:26:00       2.75     17850.0  United Kingdom  
3 2010-12-01 08:26:00       3.39     17850.0  United Kingdom  
4 2010-12-01 08:26:00       3.39     17850.0  United Kingdom  


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [8]:
df.isnull().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

# Limpieza del dataset

In [ ]:
# Eliminamos filas con nulos en 'CustomerID' y 'Description'
df.dropna(axis= 0, subset=['CustomerID', 'Description'], inplace=True)
print(f"Dimensiones tras eliminar nulos en CustomerID y Description: {df.shape}")

Dimensiones tras eliminar nulos en CustomerID y Description: (406829, 8)


In [12]:
# Convertimos 'CustomerID' a entero
df['CustomerID'] = df['CustomerID'].astype(int)

In [13]:
# Eliminamos transacciones con cantidad negativa o cero
df = df[df['Quantity'] > 0]
print(f"Dimensiones finales del dataset limpio: {df.shape}")

Dimensiones finales del dataset limpio: (397924, 8)


# Descripcion del dataset

In [15]:
# Número de transacciones (facturas únicas)
num_transacciones = df['InvoiceNo'].nunique()
print(f"Número de transacciones únicas: {num_transacciones}")

Número de transacciones únicas: 18536


In [16]:
# Número de ítems únicos
num_items = df['Description'].nunique()
print(f"Número de ítems únicos: {num_items}")

Número de ítems únicos: 3877


In [17]:
# Número de clientes únicos
num_clientes = df['CustomerID'].nunique()